# Лабораторная работа 6

   по теме
                 
   **Прогноз успеха фильмов по обзорам**
   
   ****

   Выполнил студент

   Группы БСТ1801

   Харатишвили Заза

In [63]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential

In [64]:
# Загружаем датасет
from keras.datasets import imdb
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=10000)
# Обьединяем данные в доль одной оси
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)
print("Количество оценок: ", len(targets))
print("Количество обзоров: ", len(data))

Количество оценок:  50000
Количество обзоров:  50000


In [65]:
print("Категории:", np.unique(targets)) # Положительный или отрицательный обзоры
print("Количество уникальных слов:", len(np.unique(np.hstack(data))))
length = [len(i) for i in data]
print("Средний размер обзора:", np.mean(length))
print("Стандартное отклонение:", round(np.std(length)))

Категории: [0 1]
Количество уникальных слов: 9998
Средний размер обзора: 234.75892
Стандартное отклонение: 173


In [66]:
# Посмотрим на первый обзор из датасета
print("Номер:", targets[1])
print(data[1])

# Преобразуем индексы в слова
index = imdb.get_word_index() # Получаем слова и их индексы
reverse_index = dict([(value, key) for (key, value) in index.items()])
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[1]] ) # Заменим неизвестные слова на #
print("\n Текст обзора: ", decoded)
print("\n Форма первого обзора: ", tf.shape(data[1]))

Номер: 0
[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 8255, 2, 349, 2637, 148, 605, 2, 8003, 15, 123, 125, 68, 2, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 2, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 2, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]

 Текст обзора:  # big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and

In [67]:
# Подготовим данные
# Заполним нулями каждый обзор, чтобы размер был 10000 чисел
# Это необходимо, потому что каждый элемент входных данных должен иметь одинаковый размер
print("/n Форма обзора: ",tf.shape(data[1]))

def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

data = vectorize(data)
print("/n Преобразованный обзор: ", data[1])
print("/n Форма преобразованного обзора: ", tf.shape(data[1]))

targets = np.array(targets).astype("float32") # Приведем к типу float32
print("/n Оценка фильма: ", targets[1])

/n Форма обзора:  tf.Tensor([189], shape=(1,), dtype=int32)
/n Преобразованный обзор:  [0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1.
 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 

In [68]:
# Разделим датасет на обучающий и тестировочный наборы. Обучающий 40 000 обзоров, тестовый — 10 000.
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

print(tf.shape(test_x))
print(tf.shape(test_y))

# print("test x" ,test_x)
# print("test y" ,test_y)

tf.Tensor([10000 10000], shape=(2,), dtype=int32)
tf.Tensor([10000], shape=(1,), dtype=int32)


In [71]:
# Создадим и обучим модель 
model = Sequential()
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

# Компилируем модель
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["acc"])

results = model.fit(train_x, train_y, epochs= 2, batch_size = 512, validation_data = (test_x, test_y))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 50)                500050    
                                                                 
 dropout_4 (Dropout)         (None, 50)                0         
                                                                 
 dense_10 (Dense)            (None, 50)                2550      
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_11 (Dense)            (None, 50)                2550      
                                                                 
 dense_12 (Dense)            (None, 1)                 51        
                                                                 
Total params: 505,201
Trainable params: 505,201
Non-tr

In [ ]:
# Проведем оценку работы модели
print("Точность обучения:", np.mean(results.history["val_acc"]))

Напишем функцию иморта пользовательского текстового файла

In [79]:
from nltk.tokenize import RegexpTokenizer
#np.set_printoptions(threshold=np.inf)

In [80]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    # filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(tokens)

In [81]:
file = open("text.txt",'r').read()

processed_inputs = tokenize_words(file)
processed_inputs = processed_inputs.replace("ђ", "")
processed_inputs = processed_inputs.replace("њ", "")
processed_inputs = processed_inputs.replace("ќ", "")
processed_inputs = processed_inputs.replace("в", "")

chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

print(processed_inputs)

input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Общее количество символов:", input_len)
print ("Количество уникальных букв:", vocab_len)

this film was just brilliant casting location scenery story direction everyone s really suited the part they played and you could just imagine being there robert is an amazing actor and now the same being director father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also to the two little boy s that played the of norman and paul they were just brilliant children are often left out of the list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don t you think the whole s

In [82]:
processed_inputs = "this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"

index = imdb.get_word_index() # Получаем слова и их индексы
reverse_index = dict([(value, key) for (key, value) in index.items()]) # меняем местами key и value
splitted = processed_inputs.split()


encoded = []
encoded.extend( [index.get(i, 0) for i in splitted] ) # Заменим слова на индексы, неизвестные = 0

print("Массив ключей (текста обзора): ", splitted)
print("\n Массив значений (индексов слов): ", encoded)
print("\n Размеры массивов: ")
print(tf.shape(splitted))
print(tf.shape(encoded))

Массив ключей (текста обзора):  ['this', 'film', 'was', 'just', 'brilliant', 'casting', 'location', 'scenery', 'story', 'direction', "everyone's", 'really', 'suited', 'the', 'part', 'they', 'played', 'and', 'you', 'could', 'just', 'imagine', 'being', 'there', 'robert', '#', 'is', 'an', 'amazing', 'actor', 'and', 'now', 'the', 'same', 'being', 'director', '#', 'father', 'came', 'from', 'the', 'same', 'scottish', 'island', 'as', 'myself', 'so', 'i', 'loved', 'the', 'fact', 'there', 'was', 'a', 'real', 'connection', 'with', 'this', 'film', 'the', 'witty', 'remarks', 'throughout', 'the', 'film', 'were', 'great', 'it', 'was', 'just', 'brilliant', 'so', 'much', 'that', 'i', 'bought', 'the', 'film', 'as', 'soon', 'as', 'it', 'was', 'released', 'for', '#', 'and', 'would', 'recommend', 'it', 'to', 'everyone', 'to', 'watch', 'and', 'the', 'fly', 'fishing', 'was', 'amazing', 'really', 'cried', 'at', 'the', 'end', 'it', 'was', 'so', 'sad', 'and', 'you', 'know', 'what', 'they', 'say', 'if', 'you', 

In [83]:
# Подготовим данные
# Заполним нулями каждый обзор, чтобы размер был 10000 чисел
# Это необходимо, потому что каждый элемент входных данных должен иметь одинаковый размер
print("Форма до изменения: ", tf.shape(encoded))
# def vectorize(sequences, dimension = 10000):
#     results = np.zeros(dimension)
#     for i, sequence in enumerate(sequences):
#         results[sequence] = 1
#     return results

def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

encoded = vectorize(encoded)

# print(encoded)
print("Форма после изменения: ",tf.shape(encoded))

encoded = np.array(encoded)
test_y = np.array(train_y)

Форма до изменения:  tf.Tensor([217], shape=(1,), dtype=int32)
Форма после изменения:  tf.Tensor([  217 10000], shape=(2,), dtype=int32)


In [84]:
print(model.predict(encoded, verbose=0))

[[0.47530785]
 [0.5246063 ]
 [0.49198613]
 [0.48156926]
 [0.24912474]
 [0.47203526]
 [0.548473  ]
 [0.47812507]
 [0.38059187]
 [0.5181014 ]
 [0.5322556 ]
 [0.4647102 ]
 [0.54704386]
 [0.55995226]
 [0.5260196 ]
 [0.52679753]
 [0.6825626 ]
 [0.5120189 ]
 [0.476933  ]
 [0.42886323]
 [0.48156926]
 [0.24381095]
 [0.49311718]
 [0.56722605]
 [0.49355954]
 [0.5320968 ]
 [0.48462504]
 [0.5452987 ]
 [0.6025208 ]
 [0.47966853]
 [0.5120189 ]
 [0.51897544]
 [0.55995226]
 [0.5430336 ]
 [0.49311718]
 [0.29060405]
 [0.5320968 ]
 [0.5251843 ]
 [0.4820828 ]
 [0.46419317]
 [0.55995226]
 [0.5430336 ]
 [0.43424395]
 [0.34206855]
 [0.4883548 ]
 [0.63332546]
 [0.52025396]
 [0.44780377]
 [0.53939533]
 [0.55995226]
 [0.48049864]
 [0.56722605]
 [0.49198613]
 [0.47810802]
 [0.44098338]
 [0.5339108 ]
 [0.42543894]
 [0.47530785]
 [0.5246063 ]
 [0.55995226]
 [0.73950726]
 [0.6431748 ]
 [0.48318964]
 [0.55995226]
 [0.5246063 ]
 [0.48793918]
 [0.56812507]
 [0.5095269 ]
 [0.49198613]
 [0.48156926]
 [0.24912474]
 [0.52

## Вывод

В результате выполнения лабораторной работы можно сделать вывод о том, что 